In [136]:
import pandas as pd
import numpy as np
import math
import os
import json

import viewdb_to_bq  as x

import incident_data_pb2 as pb2_incident


from google.protobuf.timestamp_pb2 import Timestamp

from datetime import datetime ,timezone

# Parameter

In [137]:
view_name="xyz_incident"
view_name_id="incident_id"
datetimeCols=["open_datetime","close_datetime"]

projectId="pongthorn"
main_dataset_id='SMartDataAnalytics'
table_name="incident"

dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
dt_imported

datetime.datetime(2024, 1, 16, 17, 16, 55)

# Init Const and Variable 

# BQ-Storage-API Data Transformation

In [138]:
df=pd.read_csv("incident_bq-storage-api.csv")

In [139]:
print("Add timestamp import")
dtimestamp = Timestamp()
dtimestamp.FromDatetime(dt_imported)
update_at_micro_timestampe =dtimestamp.ToMicroseconds()
df['update_at']=update_at_micro_timestampe 


Add timestamp import


In [140]:
print("Change action type")

def change_action_merge_to_bq_storage_api(x):
    if x=="added" or x=="changed":
        return  "UPSERT"
    else:
        return "DELETE"   
df["_CHANGE_TYPE"]=df['action'].apply(change_action_merge_to_bq_storage_api)
df=df.drop(columns=['action'])

Change action type


In [141]:
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     2 non-null      int64 
 1   inventory_id    2 non-null      int64 
 2   incident_type   2 non-null      object
 3   service_type    2 non-null      object
 4   severity        2 non-null      object
 5   status          2 non-null      object
 6   open_datetime   2 non-null      object
 7   close_datetime  1 non-null      object
 8   update_at       2 non-null      int64 
 9   _CHANGE_TYPE    2 non-null      object
dtypes: int64(3), object(7)
memory usage: 288.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4306,993,Software,Request,Critical,In Progress,2023-01-11 02:45,2024-01-16 16:00,1705425415000000,UPSERT
1,4308,19516,General Incident,Incident,Critical,Open,2024-01-16 16:00,NaN,1705425415000000,UPSERT


# Spit data into UPSERT and DELETE 

In [126]:
dfUpsert=df.query("_CHANGE_TYPE=='UPSERT'")
dfUpsert.info()
dfUpsert.tail()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     2 non-null      int64 
 1   inventory_id    2 non-null      int64 
 2   incident_type   2 non-null      object
 3   service_type    2 non-null      object
 4   severity        2 non-null      object
 5   status          2 non-null      object
 6   open_datetime   2 non-null      object
 7   close_datetime  1 non-null      object
 8   update_at       2 non-null      int64 
 9   _CHANGE_TYPE    2 non-null      object
dtypes: int64(3), object(7)
memory usage: 176.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4306,993,Software,Request,Critical,In Progress,2023-01-11 02:45,2024-01-16 16:00,1705425102000000,UPSERT
1,4308,19516,General Incident,Incident,Critical,Open,2024-01-16 16:00,NaN,1705425102000000,UPSERT


In [127]:
dfDelete=df.query("_CHANGE_TYPE=='DELETE'")
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     0 non-null      int64 
 1   inventory_id    0 non-null      int64 
 2   incident_type   0 non-null      object
 3   service_type    0 non-null      object
 4   severity        0 non-null      object
 5   status          0 non-null      object
 6   open_datetime   0 non-null      object
 7   close_datetime  0 non-null      object
 8   update_at       0 non-null      int64 
 9   _CHANGE_TYPE    0 non-null      object
dtypes: int64(3), object(7)
memory usage: 0.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE


# Upsert dataframe tranformation

# Timezone and UTC Convert
## If you convert any time of any tz to timestampe for converting to Microseconds , it wll turn into UTC  so DateTime is UTC
* to_char((abc.incident_datetime AT TIME ZONE 'Asia/Bangkok'::text),
           'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* to_char((abc.incident_datetime AT TIME ZONE 'UTC'::text),
               'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* https://www.epochconverter.com/




## WorkAround Sol: null dattime is replaced with 0(GMT:1-1-1970 12:00:00 AM)

In [128]:
if dfUpsert.empty==False:
    print("Convert strng to datetime and microseconds accordingly")

    def convert_string_to_datetime_timestamp_microseconds (dt_str):
        if isinstance(dt_str, str):
            dt=datetime.strptime(dt_str,"%Y-%m-%d %H:%M")
            
            x_timestamp = Timestamp()
            x_timestamp.FromDatetime(dt)
            micro_x =x_timestamp.ToMicroseconds()
            
            return micro_x
        else:
            return np.nan
    #        
    
    for d in datetimeCols:
        #Check whick column contain null value if so, convert float64 to int 32"
        dfUpsert[d]=dfUpsert[d].apply(convert_string_to_datetime_timestamp_microseconds)
        dfUpsert[d]=dfUpsert[d].astype('Int64')
        
# #         # handle null datetime value(workaround) for 2.0 but it seem to support in 3.0
        # 1970-01-01 00:00:00 UTC
        dfUpsert[d] = dfUpsert[d].fillna(0)
        
dfUpsert.info()
dfUpsert

Convert strng to datetime and microseconds accordingly
<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     2 non-null      int64 
 1   inventory_id    2 non-null      int64 
 2   incident_type   2 non-null      object
 3   service_type    2 non-null      object
 4   severity        2 non-null      object
 5   status          2 non-null      object
 6   open_datetime   2 non-null      Int64 
 7   close_datetime  2 non-null      Int64 
 8   update_at       2 non-null      int64 
 9   _CHANGE_TYPE    2 non-null      object
dtypes: Int64(2), int64(3), object(5)
memory usage: 180.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4306,993,Software,Request,Critical,In Progress,1673405100000000,1705420800000000,1705425102000000,UPSERT
1,4308,19516,General Incident,Incident,Critical,Open,1705420800000000,0,1705425102000000,UPSERT


# Delete dataframe tranformation

In [129]:
dfDelete=dfDelete[[view_name_id,"_CHANGE_TYPE"]]
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   incident_id   0 non-null      int64 
 1   _CHANGE_TYPE  0 non-null      object
dtypes: int64(1), object(1)
memory usage: 0.0+ bytes


,incident_id,_CHANGE_TYPE


# Write Json File to ingest to BQ by Buffer Protocol

In [130]:
if  dfUpsert.empty==False:
    upsert_json_file="incident_upsert.json"
    print(upsert_json_file)
    json_file_path=os.path.join(upsert_json_file)

    json_incident_data = json.loads(dfUpsert.to_json(orient = 'records'))
    with open(upsert_json_file, "w") as outfile:
        json.dump(json_incident_data, outfile)

incident_upsert.json


In [131]:
if  dfDelete.empty==False:
    delete_json_file="incident_delete.json"
    print(delete_json_file)
    json_file_path=os.path.join(delete_json_file)
    json_incident_data = json.loads(dfDelete.to_json(orient = 'records'))
    with open(json_file_path, "w") as outfile:
        json.dump(json_incident_data, outfile)


# BufferProto to BQ 

In [132]:
def get_data_pb2(view_name):
    
    x_data_pb2=None
    if view_name == "xyz_incident": 
        x_data_pb2=pb2_incident.IncidentData()
    else:
        raise Exception("No specified view name to get data pb2")
        
    return x_data_pb2

In [133]:
listColumns= df.columns.tolist()
print(listColumns)

['incident_id', 'inventory_id', 'incident_type', 'service_type', 'severity', 'status', 'open_datetime', 'close_datetime', 'update_at', '_CHANGE_TYPE']


In [134]:
pb2_data=get_data_pb2(view_name)
print(pb2_data.DESCRIPTOR)

In [135]:
if os.path.exists(upsert_json_file):
    print("OK")
    x.write_json_to_bq(listColumns=listColumns,
                              x_data_pb2=pb2_data,
                              json_data_file=upsert_json_file,
                              project_name=projectId,
                              dataset_name=main_dataset_id,
                              table_name=table_name)
                              



OK
Rows to table: 'projects/pongthorn/datasets/SMartDataAnalytics/tables/incident' have been written.
